In [1]:
from pathlib import Path
import pandas as pd

In [2]:
def flat_table(table: pd.DataFrame, city_id: int) -> pd.DataFrame:
    # makes table (date, product_name, city_id) from (date, product1, ... product_i)
    dfs = []

    cols = table.columns.tolist()[1:]
    for col in cols:
        d = table[['date', col]].rename({col: 'count'}, axis=1)
        fvi = d['count'].first_valid_index()
        d = d.iloc[fvi:].fillna(0)
        d = d.assign(product_name=col)
        
        if len(d) == 0:
            continue
        dfs.append(d)
    
    df = pd.concat(dfs, axis=0)
    df = df.assign(city_id=city_id)
    return df

In [3]:
def load_sheet(file: pd.ExcelFile, name: str) -> pd.DataFrame:
    df = pd.read_excel(file, name)
    return df

In [4]:
def sheet2table(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={'Календарь - Дата': 'date'})
    headers = df.columns.tolist()

    df = df.iloc[1:]
    df = df.assign(date=pd.to_datetime(df.date))

    # make msk table
    try:
        msk_last_col_ix = 3 + headers[3:].index('fit')
    except ValueError:
        msk_last_col_ix = 3 + headers[3:].index('fit.1')
        
    table_msk = df.iloc[:, :msk_last_col_ix]

    # make spb table
    table_spb = df.iloc[:, msk_last_col_ix:]
    assert table_spb.columns.tolist()[0].startswith('fit'), table_spb.columns.tolist()
    table_spb = pd.concat((df.date, table_spb), axis=1)

    # merge
    msk = flat_table(table_msk, city_id=2)
    spb = flat_table(table_spb, city_id=1)

    table = pd.concat((msk, spb))
    # ".1" adds cause of duplicating columns names
    drop_dot1 = lambda s: s[:-2] if s.endswith('.1') else s
    table = table.assign(product_name=table.product_name.apply(drop_dot1))
    
    return table

---
# Внешние факторы

In [5]:
# 4 page excel
file = pd.ExcelFile('data/raw/Внешние факторы_v121222_clear.xlsx')
sheets = file.sheet_names
sheets

['orders_count',
 'new_orders_count',
 'custom_orders_rate',
 'discounts',
 'boxes_per_delivery',
 'fooddays_per_order']

In [6]:
root = Path('data/processed/')

In [7]:
for sheet in sheets:
    print(sheet)
    df = load_sheet(file, sheet)
    df = sheet2table(df)
    df.to_csv(root / f'{sheet}.csv', index=False)

orders_count
new_orders_count
custom_orders_rate
discounts
boxes_per_delivery
fooddays_per_order


In [8]:
# postprocess

fooddays_per_order = pd.read_csv(root / 'fooddays_per_order.csv')
fooddays_per_order.sample(4)

,date,count,product_name,city_id
44784,2020-06-26,2.94,daily,1
8210,2022-07-21,0,daily plus,2
41448,2022-10-12,2.55,power,1
61950,2022-03-24,0,dsv,1


In [20]:
t = fooddays_per_order
val = '   ,'
len(t[t['count'] == val]) / len(t)

0.01228431104398806

In [21]:
fooddays_per_order = fooddays_per_order[fooddays_per_order['count'] != val]
fooddays_per_order.to_csv(root / 'fooddays_per_order.csv', index=False)

# merge to one table